In [2]:
import wandb
import os
import pytorch_lightning as pl
import torch
import numpy as np
from legendre.models.spline_cnode import SplineCNODEClass
from legendre.models.cnode_ext import CNODExtClassification, CNODExt
from legendre.models.node_ext import NODExtClassification, NODExt
from legendre.models.node_mod import NODEClassification, NODE
from legendre.models.rnn import RNNClassification, RNN
from legendre.models.hippo import HIPPO, HippoClassification
from legendre.data_utils.simple_path_utils import SimpleTrajDataModule
from legendre.data_utils.pMNIST_utils import pMNISTDataModule
from legendre.data_utils.character_utils import CharacterTrajDataModule
from legendre.data_utils.mimic_utils import MIMICDataModule
from legendre.data_utils.lorenz_utils import LorenzDataModule
from legendre.train_scripts.classif import get_init_model
from sklearn.metrics import accuracy_score, roc_auc_score
from pytorch_lightning.core.saving import _load_state


def mean_dict(d):
    return {k:sum(v)/len(v) for k,v in d.items() if len(v)>0}
def std_dict(d):
    return {k:np.std(np.array(v)) for k,v in d.items() if len(v)>0}

def process_dict(d):
    m_dict = mean_dict(d)
    s_dict = std_dict(d)
    return {k: f"${v:.3f} \\pm {s_dict[k]:.3f}$" for k,v in m_dict.items() if len(k)>0}

/voyager/projects/edebrouwer/.conda/envs/orthopoly/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Classification

## SplineCNODE

In [8]:
#from urllib.parse import non_hierarchical
sweep_id = "7iaz3zez"
multivariate = None
regression_mode = True
api= wandb.Api()
sweep = api.sweep("edebrouwer/orthopoly/"+sweep_id)
runs = sweep.runs

model_cls = SplineCNODEClass
#irregular_rates = [0.3,0.4,0.5]
irregular_rates = [0.7,0.8,0.9,1.0]
acc_dict = {}

def data_cls_choice(data_type):
    if data_type == "SimpleTraj":
        data_cls = SimpleTrajDataModule(**model.hparams)
    elif data_type == "pMNIST":
        data_cls = pMNISTDataModule(**model.hparams)
    elif data_type == "Character":
        data_cls = CharacterTrajDataModule(**model.hparams)
    elif data_type == "Lorenz":
        data_cls = LorenzDataModule(**model.hparams)
    return data_cls


for irregular_rate in irregular_rates:

    accs_ = []
    
    run_sub = [r for r in runs if (r.config["irregular_rate"]==irregular_rate) and (r.config.get("multivariate",None)==multivariate) and (r.config.get("regression_mode",False)==regression_mode)]

    for run in run_sub:
        fname = [f.name for f in run.files() if "ckpt" in f.name][0]
        run.file(fname).download(replace = True, root = ".")
        checkpoint = torch.load(fname, map_location=lambda storage, loc: storage)
        hparams = checkpoint["hyper_parameters"]
        model = model_cls.load_from_checkpoint(fname,num_dims = hparams.get("num_dims",1))
        os.remove(fname)
        dataset = data_cls_choice(model.hparams.data_type)
        dataset.prepare_data()
        
        trainer = pl.Trainer(gpus=1)
        outputs = trainer.predict(model,dataset.test_dataloader())

        preds = torch.cat([x["preds"] for x in outputs])
        Y = torch.cat([x["Y"] for x in outputs])
        T = torch.cat([x["T"] for x in outputs])
        labels = torch.cat([x["labels"] for x in outputs])
        
        if checkpoint["hyper_parameters"].get("regression_mode",False):
            accuracy = torch.nn.MSELoss()(preds,labels)
        else:
            if len(preds.shape)>1:
                preds = torch.nn.functional.softmax(preds, dim=-1).argmax(-1)
                accuracy = accuracy_score(labels.long().cpu().numpy(), preds.cpu().numpy())
            else:
                accuracy = roc_auc_score(labels.long().cpu().numpy(),preds.cpu().numpy())
        #print(accuracy)
        
        accs_.append(accuracy)
        
    acc_dict[f"{irregular_rate}"]=accs_
    print(acc_dict)
    print(process_dict(acc_dict))

Pre-computing ODE Projection embeddings....


  0%|          | 0/20 [00:00<?, ?it/s]/voyager/projects/edebrouwer/.conda/envs/orthopoly/lib/python3.9/site-packages/torchdiffeq/_impl/fixed_adams.py:220: UserWarning: Functional iteration did not converge. Solution may be incorrect.
  warnings.warn('Functional iteration did not converge. Solution may be incorrect.')
100%|██████████| 20/20 [00:08<00:00,  2.29it/s]
/voyager/projects/edebrouwer/.conda/envs/orthopoly/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:445: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
/voyager/projects/edebrouwer/.conda/envs/orthopoly/lib/python3.9/site-packages/pytorch_lightning/loops/utilities.py:93: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: True (cuda), us

Predicting DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 26.25it/s] 
Pre-computing ODE Projection embeddings....


100%|██████████| 20/20 [00:08<00:00,  2.33it/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Predicting DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 25.50it/s] 
Pre-computing ODE Projection embeddings....


...

Predicting DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 23.09it/s] 
Pre-computing ODE Projection embeddings....


100%|██████████| 20/20 [00:08<00:00,  2.35it/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Predicting DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 28.85it/s] 
Pre-computing ODE Projection embeddings....


100%|██████████| 20/20 [00:08<00:00,  2.35it/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Predicting DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 27.50it/s] 
{'0.7': [tensor(0.0771), tensor(0.0473), tensor(0.0404)], '0.8': [tensor(0.0594), tensor(0.0383), tensor(0.0475)], '0.9': [tensor(0.0369), tensor(0.0215), tensor(0.0139)], '1.0': [tensor(9.7547e-07), tensor(1.8181e-07), tensor(1.1121e-08)]}
{'0.7': '$0.055 \\pm 0.016$', '0.8': '$0.048 \\pm 0.009$', '0.9': '$0.024 \\pm 0.010$', '1.0': '$0.000 \\pm 0.000$'}


## CNODExt

In [5]:
sweep_id = "sm9uua5w"
regression_mode = True
api= wandb.Api()
sweep = api.sweep("edebrouwer/orthopoly/"+sweep_id)
#init_sweep_id = "edebrouwer/orthopoly/t77q79lw"
runs = sweep.runs

model_cls = CNODExtClassification
init_model_cls = CNODExt
#irregular_rates = [0.3, 0.4, 0.5]
irregular_rates = [0.7, 0.8, 0.9, 1.0]
device = torch.device("cuda")
seed = 421
acc_dict = {}

def data_cls_choice(data_type, init_model):
    if data_type == "SimpleTraj":
        data_cls = SimpleTrajDataModule(**init_model.hparams)
    elif data_type == "pMNIST":
        data_cls = pMNISTDataModule(**init_model.hparams)
    elif data_type == "Character":
        data_cls = CharacterTrajDataModule(**init_model.hparams)
    elif data_type == "MIMIC":
        data_cls = MIMICDataModule(**init_model.hparams)
    elif data_type == "Lorenz":
        data_cls = LorenzDataModule(**init_model.hparams)
    return data_cls

for irregular_rate in irregular_rates:
    
    accs_ = []
    run_sub = [r for r in runs if (r.config["irregular_rate"]==irregular_rate)   and  (r.config.get("regression_mode",False)==regression_mode) ]

    for run in run_sub:
        fname = [f.name for f in run.files() if "ckpt" in f.name][0]
        run.file(fname).download(replace = True, root = ".")
         
        checkpoint = torch.load(
        fname, map_location=lambda storage, loc: storage)
        checkpoint["hyper_parameters"].pop("callbacks", None)
        checkpoint["hyper_parameters"].pop("logger", None)
        checkpoint["hyper_parameters"].pop("wandb_id_file_path", None)
        checkpoint.pop("callbacks",None)
        checkpoint_ = {"state_dict":checkpoint["state_dict"],"hyper_parameters":checkpoint["hyper_parameters"]}
        checkpoint_["hyper_parameters"].pop("init_model",None)

        init_model = get_init_model(checkpoint["hyper_parameters"]["init_sweep_id"],init_model_cls,checkpoint["hyper_parameters"]["irregular_rate"],checkpoint["hyper_parameters"]["seed"],checkpoint["hyper_parameters"].get("multivariate", False))
        
        model = _load_state(model_cls, checkpoint_,init_model = init_model).to(device)
               
        os.remove(fname)
        
        dataset = data_cls_choice(model.hparams.data_type, init_model = model)
        dataset.set_test_only()
        dataset.prepare_data()
    
        trainer = pl.Trainer(gpus=1,logger = None)
        
        outputs = trainer.predict(model,dataset.test_dataloader())

        preds = torch.cat([x["preds"] for x in outputs])
        Y = torch.cat([x["Y"] for x in outputs])
        T = torch.cat([x["T"] for x in outputs])
        labels = torch.cat([x["labels"] for x in outputs])
        
        if checkpoint["hyper_parameters"].get("regression_mode",False):
            accuracy = torch.nn.MSELoss()(preds,labels)
        else:
            if len(preds.shape)>1:
                preds = torch.nn.functional.softmax(preds, dim=-1).argmax(-1)
                accuracy = accuracy_score(labels.long().cpu().numpy(), preds.cpu().numpy())
            else:
                accuracy = roc_auc_score(labels.long().cpu().numpy(),preds.cpu().numpy())
        
        accs_.append(accuracy)
    
    acc_dict[f"{irregular_rate}"]=accs_
print(acc_dict)
print(process_dict(acc_dict))

/voyager/projects/edebrouwer/.conda/envs/orthopoly/lib/python3.9/site-packages/pytorch_lightning/utilities/parsing.py:261: UserWarning: Attribute 'init_model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['init_model'])`.
  rank_zero_warn(


Pre-computing ODE Projection embeddings....


  0%|          | 0/20 [00:00<?, ?it/s]/voyager/projects/edebrouwer/.conda/envs/orthopoly/lib/python3.9/site-packages/torchdiffeq/_impl/fixed_adams.py:220: UserWarning: Functional iteration did not converge. Solution may be incorrect.
  warnings.warn('Functional iteration did not converge. Solution may be incorrect.')
100%|██████████| 20/20 [00:07<00:00,  2.62it/s]
/voyager/projects/edebrouwer/.conda/envs/orthopoly/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:445: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
/voyager/projects/edebrouwer/.conda/envs/orthopoly/lib/python3.9/site-packages/pytorch_lightning/loops/utilities.py:93: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: True (cuda), us

Predicting: 0it [00:00, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f26dec73ca0>
Traceback (most recent call last):
  File "/voyager/projects/edebrouwer/.conda/envs/orthopoly/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1481, in __del__
    self._shutdown_workers()
  File "/voyager/projects/edebrouwer/.conda/envs/orthopoly/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1412, in _shutdown_workers
    if not self._shutdown:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_shutdown'
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f26dec73ca0>
Traceback (most recent call last):
  File "/voyager/projects/edebrouwer/.conda/envs/orthopoly/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1481, in __del__
    self._shutdown_workers()
  File "/voyager/projects/edebrouwer/.conda/envs/orthopoly/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1412, in _shutdown_

...

Predicting DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 24.51it/s] 


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f26dec73ca0>
Traceback (most recent call last):
  File "/voyager/projects/edebrouwer/.conda/envs/orthopoly/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1481, in __del__
    self._shutdown_workers()
  File "/voyager/projects/edebrouwer/.conda/envs/orthopoly/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1412, in _shutdown_workers
    if not self._shutdown:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_shutdown'
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f26dec73ca0>
Traceback (most recent call last):
  File "/voyager/projects/edebrouwer/.conda/envs/orthopoly/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1481, in __del__
    self._shutdown_workers()
  File "/voyager/projects/edebrouwer/.conda/envs/orthopoly/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1412, in _shutdown_

Pre-computing ODE Projection embeddings....


100%|██████████| 20/20 [00:07<00:00,  2.63it/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Predicting DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 23.04it/s] 
{'0.7': [tensor(0.0104), tensor(0.0147), tensor(0.0120)], '0.8': [tensor(0.0082), tensor(0.0024), tensor(0.0033)], '0.9': [tensor(0.0027), tensor(0.0013), tensor(0.0005)], '1.0': [tensor(1.3792e-05), tensor(2.9344e-06), tensor(1.8820e-05)]}
{'0.7': '$0.012 \\pm 0.002$', '0.8': '$0.005 \\pm 0.003$', '0.9': '$0.002 \\pm 0.001$', '1.0': '$0.000 \\pm 0.000$'}


## Hippo

In [11]:
sweep_id = "yrex8yy3"
multivariate = None
regression_mode = True
api= wandb.Api()
sweep = api.sweep("edebrouwer/orthopoly/"+sweep_id)
runs = sweep.runs

model_cls = HIPPO

#irregular_rates = [0.3, 0.4, 0.5]
irregular_rates= [0.7,0.8,0.9,1.0]
seed = 421
acc_dict = {}

def data_cls_choice(data_type):
    if data_type == "SimpleTraj":
        data_cls = SimpleTrajDataModule(**model.hparams)
    elif data_type == "pMNIST":
        data_cls = pMNISTDataModule(**model.hparams)
    elif data_type == "Character":
        data_cls = CharacterTrajDataModule(**model.hparams)
    elif data_type == "Lorenz":
        data_cls = LorenzDataModule(**model.hparams)
    elif data_type == "MIMIC":
        data_cls = MIMICDataModule(**model.hparams)
    return data_cls

for irregular_rate in irregular_rates:
    
    accs_ = []
    run_sub = [r for r in runs if (r.config["irregular_rate"]==irregular_rate) and (r.config.get("multivariate",None)==multivariate) and (r.config.get("regression_mode",False)==regression_mode)]

    for run in run_sub:
        fname = [f.name for f in run.files() if "ckpt" in f.name][0]
        run.file(fname).download(replace = True, root = ".")
        checkpoint = torch.load(fname, map_location=lambda storage, loc: storage)
        hparams = checkpoint["hyper_parameters"]
        model = model_cls.load_from_checkpoint(fname,num_dims = hparams.get("num_dims",1))
        os.remove(fname)
        dataset = data_cls_choice(model.hparams.data_type)
        dataset.prepare_data()
        
        trainer = pl.Trainer(gpus=1)
        outputs = trainer.predict(model,dataset.test_dataloader())

        preds = torch.cat([x["preds"] for x in outputs])
        Y = torch.cat([x["Y"] for x in outputs])
        T = torch.cat([x["T"] for x in outputs])
        labels = torch.cat([x["labels"] for x in outputs])
        
        if checkpoint["hyper_parameters"].get("regression_mode",False):
            accuracy = torch.nn.MSELoss()(preds,labels)
        else:
            if len(preds.shape)>1:
                preds = torch.nn.functional.softmax(preds, dim=-1).argmax(-1)
                accuracy = accuracy_score(labels.long().cpu().numpy(), preds.cpu().numpy())
            else:
                accuracy = roc_auc_score(labels.long().cpu().numpy(),preds.cpu().numpy())
        #print(accuracy)
        
        accs_.append(accuracy)
    
    acc_dict[f"{irregular_rate}"]=accs_
print(acc_dict)
print(process_dict(acc_dict))

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f211e253550>
Traceback (most recent call last):
  File "/voyager/projects/edebrouwer/.conda/envs/orthopoly/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1481, in __del__
    self._shutdown_workers()
  File "/voyager/projects/edebrouwer/.conda/envs/orthopoly/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1412, in _shutdown_workers
    if not self._shutdown:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_shutdown'
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f211e253550>
Traceback (most recent call last):
  File "/voyager/projects/edebrouwer/.conda/envs/orthopoly/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1481, in __del__
    self._shutdown_workers()
  File "/voyager/projects/edebrouwer/.conda/envs/orthopoly/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1412, in _shutdown_

Predicting DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 18.58it/s]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Predicting DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 17.61it/s]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


...

Predicting DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 20.78it/s]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Predicting DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 16.87it/s]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Predicting DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 16.80it/s]
{'0.7': [tensor(0.1830), tensor(0.2075), tensor(0.2013)], '0.8': [tensor(0.1549), tensor(0.1711), tensor(0.1585)], '0.9': [tensor(0.0903), tensor(0.1008), tensor(0.1036)], '1.0': [tensor(1.4283e-07), tensor(1.5542e-07), tensor(2.3845e-06)]}
{'0.7': '$0.197 \\pm 0.010$', '0.8': '$0.161 \\pm 0.007$', '0.9': '$0.098 \\pm 0.006$', '1.0': '$0.000 \\pm 0.000$'}


## Hippo-RNN

In [10]:
sweep_id = "4xatx2ex"
device = torch.device("cuda")
multivariate = None
regression_mode = True
api= wandb.Api()
sweep = api.sweep("edebrouwer/orthopoly/"+sweep_id)
#init_sweep_id = "edebrouwer/orthopoly/t77q79lw"
runs = sweep.runs

model_cls = HippoClassification
init_model_cls = HIPPO
#irregular_rates= [1.0]
#irregular_rates = [0.3, 0.4, 0.5]#, 0.9, 1.0]
irregular_rates = [0.7, 0.8, 0.9, 1.0]#, 0.9, 1.0]
seed = 421
acc_dict = {}

def data_cls_choice(data_type, init_model):
    if data_type == "SimpleTraj":
        data_cls = SimpleTrajDataModule(**init_model.hparams)
    elif data_type == "pMNIST":
        data_cls = pMNISTDataModule(**init_model.hparams)
    elif data_type == "Character":
        data_cls = CharacterTrajDataModule(**init_model.hparams)
    elif data_type == "MIMIC":
        data_cls = MIMICDataModule(**init_model.hparams)
    elif data_type == "Lorenz":
        data_cls = LorenzDataModule(**init_model.hparams)
    return data_cls

for irregular_rate in irregular_rates:
    
    accs_ = []
    run_sub = [r for r in runs if (r.config["irregular_rate"]==irregular_rate) and (r.config.get("multivariate",None)==multivariate) and (r.config.get("regression_mode",False)==regression_mode)]

    for run in run_sub:
        fname = [f.name for f in run.files() if "ckpt" in f.name][0]
        run.file(fname).download(replace = True, root = ".")
         
        checkpoint = torch.load(
        fname, map_location=lambda storage, loc: storage)
        checkpoint["hyper_parameters"].pop("callbacks", None)
        checkpoint["hyper_parameters"].pop("logger", None)
        checkpoint["hyper_parameters"].pop("wandb_id_file_path", None)
        checkpoint.pop("callbacks",None)
        checkpoint_ = {"state_dict":checkpoint["state_dict"],"hyper_parameters":checkpoint["hyper_parameters"]}
        checkpoint_["hyper_parameters"].pop("init_model",None)
        
        #input_dim = checkpoint["hyper_parameters"].pop("input_dim",None)
        #if input_dim is None:
        #    init_model = NODExt( **checkpoint["hyper_parameters"],output_dim = checkpoint["hyper_parameters"]["num_dims"])
        #else:
        #    init_model = NODExt(output_dim = input_dim, **checkpoint["hyper_parameters"])
        
        init_model = get_init_model(checkpoint["hyper_parameters"]["init_sweep_id"],init_model_cls,checkpoint["hyper_parameters"]["irregular_rate"],checkpoint["hyper_parameters"]["seed"],checkpoint["hyper_parameters"].get("multivariate", False))
        
        model = _load_state(model_cls, checkpoint_,init_model = init_model).to(device)
               
        #model = model_cls.load_from_checkpoint(fname)
        os.remove(fname)
        
        dataset = data_cls_choice(model.hparams.data_type, init_model = model)
        #dataset.pre_compute_ode = False
        dataset.set_test_only()
        dataset.prepare_data()
    
        trainer = pl.Trainer(gpus=1,logger = None)
        
        outputs = trainer.predict(model,dataset.test_dataloader())

        preds = torch.cat([x["preds"] for x in outputs])
        Y = torch.cat([x["Y"] for x in outputs])
        T = torch.cat([x["T"] for x in outputs])
        labels = torch.cat([x["labels"] for x in outputs])
        
        if checkpoint["hyper_parameters"].get("regression_mode",False):
            accuracy = torch.nn.MSELoss()(preds,labels)
        else:
            if len(preds.shape)>1:
                preds = torch.nn.functional.softmax(preds, dim=-1).argmax(-1)
                accuracy = accuracy_score(labels.long().cpu().numpy(), preds.cpu().numpy())
            else:
                accuracy = roc_auc_score(labels.long().cpu().numpy(),preds.cpu().numpy())
        #print(accuracy)
        
        accs_.append(accuracy)
    
    acc_dict[f"{irregular_rate}"]=accs_
print(acc_dict)
print(process_dict(acc_dict))

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f211e253550>
Traceback (most recent call last):
  File "/voyager/projects/edebrouwer/.conda/envs/orthopoly/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1481, in __del__
  File "/voyager/projects/edebrouwer/.conda/envs/orthopoly/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1412, in _shutdown_workers
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_shutdown'
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f211e253550>
Traceback (most recent call last):
  File "/voyager/projects/edebrouwer/.conda/envs/orthopoly/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1481, in __del__
  File "/voyager/projects/edebrouwer/.conda/envs/orthopoly/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1412, in _shutdown_workers
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_sh

Pre-computing ODE Projection embeddings....


100%|██████████| 20/20 [00:00<00:00, 84.01it/s]
/voyager/projects/edebrouwer/.conda/envs/orthopoly/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:445: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
/voyager/projects/edebrouwer/.conda/envs/orthopoly/lib/python3.9/site-packages/pytorch_lightning/loops/utilities.py:93: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Predicting DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 21.65it/s] 
Pre-computing ODE Projection embeddings....


100%|██████████| 20/20 [00:00<00:00, 87.65it/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


...

Pre-computing ODE Projection embeddings....


100%|██████████| 20/20 [00:00<00:00, 85.47it/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Predicting DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 21.92it/s] 
Pre-computing ODE Projection embeddings....


100%|██████████| 20/20 [00:00<00:00, 82.61it/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Predicting DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 19.33it/s] 
{'0.7': [tensor(0.2334), tensor(0.1934), tensor(0.1995)], '0.8': [tensor(0.1677), tensor(0.1418), tensor(0.1864)], '0.9': [tensor(0.0580), tensor(0.0970), tensor(0.1154)], '1.0': [tensor(0.0002), tensor(7.3203e-05), tensor(4.9701e-06)]}
{'0.7': '$0.209 \\pm 0.018$', '0.8': '$0.165 \\pm 0.018$', '0.9': '$0.090 \\pm 0.024$', '1.0': '$0.000 \\pm 0.000$'}


## NODExt

In [3]:
sweep_id = "ql8em3f7"
device = torch.device("cuda")
multivariate = None
regression_mode = True
api= wandb.Api()
sweep = api.sweep("edebrouwer/orthopoly/"+sweep_id)
#init_sweep_id = "edebrouwer/orthopoly/t77q79lw"
runs = sweep.runs

model_cls = NODEClassification
init_model_cls = NODE
irregular_rates = [0.3, 0.4, 0.5]
#irregular_rates = [1.0]
#irregular_rates = [0.7, 0.8, 0.9, 1.0]
seed = 421
acc_dict = {}

def data_cls_choice(data_type, init_model):
    if data_type == "SimpleTraj":
        data_cls = SimpleTrajDataModule(**init_model.hparams)
    elif data_type == "pMNIST":
        data_cls = pMNISTDataModule(**init_model.hparams)
    elif data_type == "Character":
        data_cls = CharacterTrajDataModule(**init_model.hparams)
    elif data_type == "MIMIC":
        data_cls = MIMICDataModule(**init_model.hparams)
    elif data_type == "Lorenz":
        data_cls = LorenzDataModule(**init_model.hparams)
    return data_cls

for irregular_rate in irregular_rates:
    
    accs_ = []
    run_sub = [r for r in runs if (r.config["irregular_rate"]==irregular_rate) and (r.config.get("multivariate",None)==multivariate) and (r.config.get("regression_mode",False)==regression_mode)]

    for run in run_sub:
        fname = [f.name for f in run.files() if "ckpt" in f.name][0]
        run.file(fname).download(replace = True, root = ".")
         
        checkpoint = torch.load(
        fname, map_location=lambda storage, loc: storage)
        checkpoint["hyper_parameters"].pop("callbacks", None)
        checkpoint["hyper_parameters"].pop("logger", None)
        checkpoint["hyper_parameters"].pop("wandb_id_file_path", None)
        checkpoint.pop("callbacks",None)
        checkpoint_ = {"state_dict":checkpoint["state_dict"],"hyper_parameters":checkpoint["hyper_parameters"]}
        checkpoint_["hyper_parameters"].pop("init_model",None)
        
        #input_dim = checkpoint["hyper_parameters"].pop("input_dim",None)
        #if input_dim is None:
        #    init_model = NODExt( **checkpoint["hyper_parameters"],output_dim = checkpoint["hyper_parameters"]["num_dims"])
        #else:
        #    init_model = NODExt(output_dim = input_dim, **checkpoint["hyper_parameters"])
        
        init_model = get_init_model(checkpoint["hyper_parameters"]["init_sweep_id"],init_model_cls,checkpoint["hyper_parameters"]["irregular_rate"],checkpoint["hyper_parameters"]["seed"],checkpoint["hyper_parameters"].get("multivariate", False))
        
        model = _load_state(model_cls, checkpoint_,init_model = init_model).to(device)
               
        #model = model_cls.load_from_checkpoint(fname)
        os.remove(fname)
        
        dataset = data_cls_choice(model.hparams.data_type, init_model = model)
        #dataset.pre_compute_ode = False
        dataset.set_test_only()
        dataset.prepare_data()
    
        trainer = pl.Trainer(gpus=1,logger = None)
        
        outputs = trainer.predict(model,dataset.test_dataloader())

        preds = torch.cat([x["preds"] for x in outputs])
        Y = torch.cat([x["Y"] for x in outputs])
        T = torch.cat([x["T"] for x in outputs])
        labels = torch.cat([x["labels"] for x in outputs])
        
        if checkpoint["hyper_parameters"].get("regression_mode",False):
            accuracy = torch.nn.MSELoss()(preds,labels)
        else:
            if len(preds.shape)>1:
                preds = torch.nn.functional.softmax(preds, dim=-1).argmax(-1)
                accuracy = accuracy_score(labels.long().cpu().numpy(), preds.cpu().numpy())
            else:
                accuracy = roc_auc_score(labels.long().cpu().numpy(),preds.cpu().numpy())
        #print(accuracy)
        
        accs_.append(accuracy)
    
    acc_dict[f"{irregular_rate}"]=accs_
print(acc_dict)
print(process_dict(acc_dict))

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7efc56285c10>
Traceback (most recent call last):
  File "/voyager/projects/edebrouwer/.conda/envs/orthopoly/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1481, in __del__
    self._shutdown_workers()
  File "/voyager/projects/edebrouwer/.conda/envs/orthopoly/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1412, in _shutdown_workers
    if not self._shutdown:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_shutdown'
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7efc56285c10>
Traceback (most recent call last):
  File "/voyager/projects/edebrouwer/.conda/envs/orthopoly/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1481, in __del__
    self._shutdown_workers()
  File "/voyager/projects/edebrouwer/.conda/envs/orthopoly/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1412, in _shutdown_

Pre-computing ODE Projection embeddings....


  0%|          | 0/20 [00:00<?, ?it/s]/voyager/projects/edebrouwer/.conda/envs/orthopoly/lib/python3.9/site-packages/torchdiffeq/_impl/fixed_adams.py:220: UserWarning: Functional iteration did not converge. Solution may be incorrect.
  warnings.warn('Functional iteration did not converge. Solution may be incorrect.')
 45%|████▌     | 9/20 [00:08<00:10,  1.08it/s]

## RNN

In [15]:
sweep_id = "1vc3k85v"
device = torch.device("cuda")
multivariate = True
regression_mode = False
api= wandb.Api()
sweep = api.sweep("edebrouwer/orthopoly/"+sweep_id)
#init_sweep_id = "edebrouwer/orthopoly/t77q79lw"
runs = sweep.runs

model_cls = RNNClassification
init_model_cls = RNN
#irregular_rates = [0.6, 0.7, 0.8, 0.9, 1.0]
irregular_rates = [0.3, 0.4, 0.5, 0.6]
seed = 421
acc_dict = {}

def data_cls_choice(data_type, init_model):
    if data_type == "SimpleTraj":
        data_cls = SimpleTrajDataModule(**init_model.hparams)
    elif data_type == "pMNIST":
        data_cls = pMNISTDataModule(**init_model.hparams)
    elif data_type == "Character":
        data_cls = CharacterTrajDataModule(**init_model.hparams)
    return data_cls

for irregular_rate in irregular_rates:
    
    accs_ = []
    run_sub = [r for r in runs if (r.config["irregular_rate"]==irregular_rate) and (r.config.get("multivariate",None)==multivariate) and (r.config.get("regression_mode",False)==regression_mode)]

    for run in run_sub:
        fname = [f.name for f in run.files() if "ckpt" in f.name][0]
        run.file(fname).download(replace = True, root = ".")
         
        checkpoint = torch.load(
        fname, map_location=lambda storage, loc: storage)
        checkpoint["hyper_parameters"].pop("callbacks", None)
        checkpoint["hyper_parameters"].pop("logger", None)
        checkpoint["hyper_parameters"].pop("wandb_id_file_path", None)
        checkpoint.pop("callbacks",None)
        checkpoint_ = {"state_dict":checkpoint["state_dict"],"hyper_parameters":checkpoint["hyper_parameters"]}
        checkpoint_["hyper_parameters"].pop("init_model",None)
        
        init_model = get_init_model(checkpoint["hyper_parameters"]["init_sweep_id"],init_model_cls,checkpoint["hyper_parameters"]["irregular_rate"],checkpoint["hyper_parameters"]["seed"],checkpoint["hyper_parameters"].get("multivariate", False))
        
        model = _load_state(model_cls, checkpoint_,init_model = init_model).to(device)
        os.remove(fname)
        
        dataset = data_cls_choice(model.hparams.data_type, init_model = model)
        dataset.prepare_data()
    
        trainer = pl.Trainer(gpus=1,logger = None)
        
        outputs = trainer.predict(model,dataset.test_dataloader())

        preds = torch.cat([x["preds"] for x in outputs])
        Y = torch.cat([x["Y"] for x in outputs])
        T = torch.cat([x["T"] for x in outputs])
        labels = torch.cat([x["labels"] for x in outputs])
        
        if len(preds.shape)>1:
            preds = torch.nn.functional.softmax(preds, dim=-1).argmax(-1)
            accuracy = accuracy_score(labels.long().cpu().numpy(), preds.cpu().numpy())
        else:
            accuracy = roc_auc_score(labels.long().cpu().numpy(),preds.cpu().numpy())
        #print(accuracy)
        
        accs_.append(accuracy)
    
    acc_dict[f"{irregular_rate}"]=accs_
print(acc_dict)

/voyager/projects/edebrouwer/.conda/envs/orthopoly/lib/python3.9/site-packages/pytorch_lightning/utilities/parsing.py:261: UserWarning: Attribute 'init_model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['init_model'])`.
  rank_zero_warn(
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f11c0f9f9d0>
Traceback (most recent call last):
  File "/voyager/projects/edebrouwer/.conda/envs/orthopoly/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1481, in __del__
    self._shutdown_workers()
  File "/voyager/projects/edebrouwer/.conda/envs/orthopoly/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1412, in _shutdown_workers
    if not self._shutdown:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_shutdown'
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f11c0f9f9d0>
Traceback

Pre-computing ODE Projection embeddings....


100%|██████████| 20/20 [00:00<00:00, 37.79it/s]


Pre-computing ODE Projection embeddings....


100%|██████████| 20/20 [00:00<00:00, 38.12it/s]
/voyager/projects/edebrouwer/.conda/envs/orthopoly/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:445: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
/voyager/projects/edebrouwer/.conda/envs/orthopoly/lib/python3.9/site-packages/pytorch_lightning/loops/utilities.py:93: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


...

Pre-computing ODE Projection embeddings....


100%|██████████| 20/20 [00:00<00:00, 52.67it/s]


Pre-computing ODE Projection embeddings....


100%|██████████| 20/20 [00:00<00:00, 52.93it/s]
/voyager/projects/edebrouwer/.conda/envs/orthopoly/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:445: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
/voyager/projects/edebrouwer/.conda/envs/orthopoly/lib/python3.9/site-packages/pytorch_lightning/loops/utilities.py:93: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Predicting DataLoader 0: 100%|██████████| 23/23 [00:00<00:00, 155.72it/s]
{'0.3': [0.883484954513646], '0.4': [0.8540937718684395], '0.5': [0.8992302309307207], '0.6': [0.9062281315605318]}


# Forecasting

In [1]:
sweep_id = "c6yoaram"
multivariate = None
api= wandb.Api()
sweep = api.sweep("edebrouwer/orthopoly/"+sweep_id)
#init_sweep_id = "edebrouwer/orthopoly/t77q79lw"
runs = sweep.runs

model_cls = NODExt
irregular_rates = [0.6, 0.7, 0.8, 0.9, 1.0]
seed = 421
acc_dict = {}

def data_cls_choice(data_type, hparams):
    if data_type == "SimpleTraj":
        data_cls = SimpleTrajDataModule(**hparams, forecast_mode = True)
    elif data_type == "pMNIST":
        data_cls = pMNISTDataModule(**hparams, forecast_mode = True)
    elif data_type == "Character":
        data_cls = CharacterTrajDataModule(**hparams, forecast_mode = True)
    return data_cls

for irregular_rate in irregular_rates:
    
    accs_ = []
    run_sub = [r for r in runs if (r.config["irregular_rate"]==irregular_rate) and (r.config.get("multivariate",None)==multivariate)]

    for run in run_sub:
        fname = [f.name for f in run.files() if "ckpt" in f.name][0]
        run.file(fname).download(replace = True, root = ".")
         
        checkpoint = torch.load(
        fname, map_location=lambda storage, loc: storage)
        checkpoint["hyper_parameters"].pop("callbacks", None)
        checkpoint["hyper_parameters"].pop("logger", None)
        checkpoint["hyper_parameters"].pop("wandb_id_file_path", None)
        checkpoint.pop("callbacks",None)
        checkpoint_ = {"state_dict":checkpoint["state_dict"],"hyper_parameters":checkpoint["hyper_parameters"]}
        checkpoint_["hyper_parameters"].pop("init_model",None)
        
        input_dim = checkpoint["hyper_parameters"].pop("input_dim",None)
                
        dataset = data_cls_choice(checkpoint["hyper_parameters"]["data_type"],checkpoint["hyper_parameters"])

        if input_dim is None:
            model = _load_state(model_cls, checkpoint_, output_dim = dataset.num_dims)

            #init_model = CNODExt( **checkpoint["hyper_parameters"],output_dim = checkpoint["hyper_parameters"]["num_dims"])
        else:
            model = _load_state(model_cls, checkpoint_, output_dim = input_dim)
            #init_model = CNODExt(output_dim = input_dim, **checkpoint["hyper_parameters"])
        
               
        #model = model_cls.load_from_checkpoint(fname)
        os.remove(fname)
        
        #dataset.pre_compute_ode = False
        dataset.prepare_data()
    
        trainer = pl.Trainer(gpus=1,logger = None)
        
        outputs = trainer.predict(model,dataset.test_dataloader())

        preds = torch.cat([x["preds"] for x in outputs])
        Y = torch.cat([x["Y"] for x in outputs])
        T = torch.cat([x["T"] for x in outputs])
        labels = torch.cat([x["labels"] for x in outputs])
        
        if len(preds.shape)>1:
            preds = torch.nn.functional.softmax(preds, dim=-1).argmax(-1)
            accuracy = accuracy_score(labels.long().cpu().numpy(), preds.cpu().numpy())
        else:
            accuracy = roc_auc_score(labels.long().cpu().numpy(),preds.cpu().numpy())
        #print(accuracy)
        
        accs_.append(accuracy)
    
    acc_dict[f"{irregular_rate}"]=accs_
print(acc_dict)

NameError: name 'wandb' is not defined